In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
data = pd.read_csv('../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')

In [ ]:
n=0
train_phe=[]
train_nom=[]
train=[]
test=[];test_phe=[];test_nom=[]
for i in range(len(data)):
    if data['Dataset_type'][i] == 'TRAIN':
        if data['Label'][i] == 'Pnemonia':
            train_phe.append(data['X_ray_image_name'][i])
        else :
            train_nom.append(data['X_ray_image_name'][i])
    else :
        if data['Label'][i] == 'Pnemonia':
            test_phe.append(data['X_ray_image_name'][i])
        else :
            test_nom.append(data['X_ray_image_name'][i])
        

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import cv2
def showdata(data,title):
    plt.figure(figsize = (12,10))
    for i in range(1,10,1):
        img = cv2.imread('../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'
                +data[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB ) 
        plt.subplot(3,3,i)
        plt.title(title)
        plt.imshow(img)
    plt.show()
showdata(train_nom,'Nomal')
showdata(train_phe,'Pnemonia')

In [ ]:
import os
output = '/kaggle/working/data/'
try:
    os.mkdir(output)

    os.mkdir(output+'/train')
    os.mkdir(output+'/test')

    os.mkdir(output+'train/Normal')
    os.mkdir(output+'train/Pnemonia')

    os.mkdir(output+'test/Normal')
    os.mkdir(output+'test/Pnemonia')
except Exception as e :
    print(e)

In [ ]:
import shutil
def builtdata(data,name,classify):
    for i in range(len(data)):
        dest = output+classify+name+data[i]
        src = '../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/'+classify +data[i]
        shutil.copyfile(src, dest)
    print('ok')

In [ ]:
builtdata(test_nom,'Normal/','test/')
builtdata(train_phe,'Pnemonia/','train/')
builtdata(train_nom,'Normal/','train/')
builtdata(test_phe,'Pnemonia/','test/')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_path = './data/train/'
# valid_path = 'data/dogs-vs-cats/valid'
test_path = './data/test'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['Pnemonia', 'Normal'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['Pnemonia', 'Normal'], batch_size=10, shuffle=False)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

img,label = next(train_batches)
def plot_img(img_arr):
    fig,axes=plt.subplots(1,10,figsize=(20,20))
    axes=axes.flatten()
    for img, ax in zip(img_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
#plot_img(img)
#print(label)

vgg16 = tf.keras.applications.vgg16.VGG16()
vgg16.summary()

model = Sequential()
for layers in vgg16.layers[:-1]:
    model.add(layers)
for layers in model.layers:
    layers.trainable = False
model.add(Dense(units=2,activation='softmax'))
model.summary()

model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
model.fit(x = train_batches, 
          steps_per_epoch = len(train_batches),
          epochs = 5,
          verbose = 2
         )

In [ ]:
predictions = model.predict(x = test_batches, steps = len(test_batches), verbose = 0)

In [ ]:
 import itertools

def plot_confusion_matrix(cm, classes,
                           normalize=False,
                           title='Confusion matrix',
                           cmap=plt.cm.Blues):
     """
     This function prints and plots the confusion matrix.
     Normalization can be applied by setting `normalize=True`.
     """
     plt.imshow(cm, interpolation='nearest', cmap=cmap)
     plt.title(title)
     plt.colorbar()
     tick_marks = np.arange(len(classes))
     plt.xticks(tick_marks, classes, rotation=45)
     plt.yticks(tick_marks, classes)

     if normalize:
         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
         print("Normalized confusion matrix")
     else:
         print('Confusion matrix, without normalization')

     print(cm)

     thresh = cm.max() / 2.
     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
         plt.text(j, i, cm[i, j],
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

     plt.tight_layout()
     plt.ylabel('True label')
     plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
cm_plot_labels = ['Phemonia','Normal']
plot_confusion_matrix(cm, classes=cm_plot_labels, title='Confusion Matrix')